In [1]:
import requests
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [2]:
import numpy as np
import json

In [3]:
##This is the initial API call, so that we do not have magic numbers in our for loop
init_college_scorecard_api = requests.get("https://api.data.gov/ed/collegescorecard/v1/schools.json?&_page=9&_per_page=50&school.online_only=0&_fields=school.name,2015.student.size&2015.student.size__range=1100..&api_key=TZzGbQIxCUsSuRlsV8kGenTldy3nkn1bAQOqjezi")

In [4]:
init_college_data = init_college_scorecard_api.json()

In [5]:
len(init_college_data["results"])

50

In [6]:
int(np.ceil(init_college_data["metadata"]["total"]/init_college_data['metadata']["per_page"]))

46

In [7]:
init_college_data['metadata']["per_page"]

50

In [8]:
###This puts the results into a dictionary, with the college name as the key, and the value as the enrollment
###For some reason, there is a json encoder error sometimes... i dont know why it's random each time this gets run
data_enrollment_dict = {}
for j in range(0,int(np.ceil(init_college_data["metadata"]["total"]/init_college_data['metadata']["per_page"]))):
    page = j
    college_scorecard_api = requests.get("https://api.data.gov/ed/collegescorecard/v1/schools.json?&_page="+str(page)+"&_per_page=50&_fields=school.name,2015.student.size&2015.student.size__range=1100..&api_key=TZzGbQIxCUsSuRlsV8kGenTldy3nkn1bAQOqjezi")
    college_data = college_scorecard_api.json()
    for i in range(len(college_data['results'])):
        data_enrollment_dict[college_data['results'][i]['school.name']]=college_data['results'][i]['2015.student.size']
#         print(college_data['results'][i]['school']['name'])
#         print(college_data['results'][i]['2015']['student']['size'])
        print(str(j)+ " loadings results...")
    
    

0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings results...
0 loadings

In [10]:
##Convert the dictionary to a Pandas Dataframe
data_df = pd.DataFrame(list(data_enrollment_dict.items()),columns = ["school_name","enrollment"])

In [11]:
data_df

In [9]:
data_enrollment_dict

In [48]:
##Reset the index (this is necessary to create a primary id for when we put this into SQL)
data_df = data_df.reset_index()

In [50]:
##do not do this unless you want a csv, i did this in case this code never works again...
data_df.to_csv("backup.csv")

In [51]:
##Convert the dataframe back into a dictionary (again necessary for SQL)
new_data_enrollment_dict = data_df.to_dict(orient="records")

In [53]:
##Putting stuff into a SQLITE database
##Most of this is copied code I dont really understand...

In [54]:
Base = declarative_base()

In [10]:
class Enrollment(Base):
    __tablename__ = "total_enrollment"
    id = Column(Integer, primary_key=True)
    school_name = Column(String)
    enrollment = Column(Integer)
    
    

NameError: name 'Base' is not defined

In [56]:
engine = create_engine("sqlite:///data_college.sqlite")
Base.metadata.create_all(engine)

In [39]:
##THIS WILL DELETE THE TABLE
##for when i screw up...
session = sessionmaker()
session.configure(bind=engine)
s = session()
Enrollment.__table__.drop(engine)

In [57]:
conn = engine.connect()

In [58]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [60]:
table_enrollment

Table('total_enrollment', MetaData(bind=Engine(sqlite:///data_college.sqlite)), Column('id', INTEGER(), table=<total_enrollment>, primary_key=True, nullable=False), Column('school_name', VARCHAR(), table=<total_enrollment>), Column('enrollment', INTEGER(), table=<total_enrollment>), schema=None)

In [59]:
table_enrollment = sqlalchemy.Table("total_enrollment", metadata, autoload=True)

In [62]:
conn.execute(table_enrollment.insert(), new_data_enrollment_dict)

In [63]:
conn.execute("select * from total_enrollment").fetchall()

[(1, 'Western Governors University', 54735),
 (2, 'Erie Community College', 10027),
 (3, 'The Art Institute of Pittsburgh-Online Division', 6883),
 (4, 'Grantham University', 9828),
 (5, 'Norco College', 8104),
 (6, 'Middle Georgia State University', 7225),
 (7, 'DeVry University-Illinois', 16211),
 (8, 'University of North Georgia', 16048),
 (9, 'Central Georgia Technical College', 6811),
 (10, 'College of Western Idaho', 6951),
 (11, 'Texas State Technical College', 8160),
 (12, 'University of Phoenix-Arizona', 129139),
 (13, 'Georgia Military College', 6840),
 (14, 'South Louisiana Community College', 5640),
 (15, 'American InterContinental University-Online', 10375),
 (16, 'University of California-Merced', 6237),
 (17, 'Folsom Lake College', 7966),
 (18, 'University of Arkansas', 21815),
 (19, 'California State University-Fullerton', 33009),
 (20, 'American River College', 26760),
 (21, 'California State University-San Bernardino', 17518),
 (22, 'California State University-Chico'